In [ ]:
%config IPCompleter.greedy=True

In [ ]:
# 匯入相關所需的模組
import os,sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from random import shuffle

from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras import utils as np_utils

In [ ]:
images=[]
labels=[]
name=[]
train_data = './characters-20'
test_data = './test'

In [ ]:
def read_images_labels(path,i):
    for file in os.listdir(path):
        abs_path=os.path.abspath(os.path.join(path,file))
        if os.path.isdir(abs_path):
            i+=1
            temp=os.path.split(abs_path)[-1]
            name.append(temp)
            read_images_labels(abs_path,i)
            amount=int(len(os.listdir(path)))
            sys.stdout.write('\r'+'>'*(i)+' '*(amount-i)+'[%s%%]'%(i*100/amount)+temp)
        else:
            if file.endswith('.jpg'):
                image = cv2.resize(cv2.imread(abs_path),(64,64))
                images.append(image)
                labels.append(i-1)
    return images, labels, name

In [ ]:
def read_main(path):
    images, labels, name = read_images_labels(path,i=0)
    images=np.array(images,dtype=np.float32)/255
    labels=np_utils.to_categorical(labels, num_classes=20)
    np.savetxt('name.txt',name,delimiter=' ',fmt="%s")
    return images, labels

In [ ]:
images,labels=read_main(train_data)
X_train,X_test,Y_train,Y_test=train_test_split(images,labels,test_size=0.1)

In [ ]:
print(images)

In [ ]:
plt.imshow(X_train[2][:,:,[2,1,0]])

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
    
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
    
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
    
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
    
model.add(Dense(20, activation='softmax'))
    

model.summary() # 秀出模型架構

In [ ]:
# 讓我們先配置一個常用的組合來作為後續優化的基準點
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
epochs=50
batch_size=64
history = model.fit(X_train, Y_train,
         batch_size=batch_size,
         epochs=epochs,
         validation_data=(X_test, Y_test),
         shuffle=True,verbose=1)

In [ ]:
print(history.history.keys())

In [ ]:
# 透過趨勢圖來觀察訓練與驗證的走向 (特別去觀察是否有"過擬合(overfitting)"的現象)

def plot_train_history(history, train_metrics, val_metrics):
    plt.plot(history.history.get(train_metrics),'-o')
    plt.plot(history.history.get(val_metrics),'-o')
    plt.ylabel(train_metrics)
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'])
    
    
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plot_train_history(history, 'loss','val_loss')

plt.subplot(1,2,2)
plot_train_history(history, 'accuracy','val_accuracy')

plt.show()

In [ ]:
file_name="model_1_"+str(epochs)+"_"+str(batch_size)
model.save(file_name+'.h5')
score=model.evaluate(X_test,Y_test,verbose=1)
print(score)

In [ ]:
image_len=len([name for name in os.listdir(test_data) if os.path.isfile(os.path.join(test_data, name))]) 
image_len

In [ ]:
def read_images(path):
    test_images = []
    for i in range(image_len):
            img = cv2.resize(cv2.imread(path+str(i+1)+'.jpg'),(64,64))
            test_images.append(img)   
    test_images=np.array(test_images,dtype=np.float32)/255.
    return test_images

In [ ]:
# 預測與比對
from keras.models import load_model

# 把訓練時val_loss最小的模型載入
model = load_model('model_1_50_256.h5')
test_img=read_images(test_data)
# 預測與比對
pred = model.predict_classes(test_img,verbose=1)
#acc = np.sum(y_pred==np.argmax(y_valtest, axis=1))/np.size(y_pred)
#print("Test accuracy = {}".format(acc))
pred

In [ ]:
nametxt=np.loadtxt('name.txt',dtype='str')

name_label=[]
name_num=0
for i in nametxt:
    name_label.append([name_num,i])
    name_num+=1
name_label

In [ ]:
new_pred=[]
for i in range(len(pred)):
    new_pred.append(name_label[pred[i]][1])
new_pred

In [ ]:
with open('test.csv','w+') as ans:
    ans.write('id,character\n')
    id=1
    for i in range(len(new_pred)):
        ans.write(str(id)+','+new_pred[i]+'\n')
        id+=1